In [0]:
# This Nb is one of the Tasks of the 251201_job_demo Job.
# The Job has 2 parameters and is configured to be triggered on file arrival to the location below.
# This Nb will get executed as soon as a new file arrives to the location below.
my_catalog = dbutils.widgets.get("catalog")
my_schema = dbutils.widgets.get("schema")
my_volume_path = f"/Volumes/{my_catalog}/{my_schema}/dataset/trigger_storage_location/"

print (my_volume_path)

In [0]:
from pyspark.sql.functions import current_timestamp, col

#
# Create customers table from customers.csv
#

# Step 1: Read csv file into a data frame
df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(my_volume_path)
    .withColumn("processing_time", current_timestamp())
    .withColumn("file_name", col("_metadata.file_path"))
)
#display(df)

# Step 2: Create a table 
df.write.mode("overwrite").saveAsTable(f"{my_catalog}.{my_schema}.bronze_customers")

In [0]:
# Post-check cell. Do not include it later in the job
spark.read.table("job_catalog.default.bronze_customers").display()
# OBS! the spark.read.table() reads a table from the catalog and saves it in a data frame